In [1]:

from dingo import MetabolicNetwork
import numpy as np

import sys
sys.path.insert(0, '../scripts/')
from load_modify_sample_utils import load_model, get_objective_functions, get_reaction_bounds, modify_model, sample_gapsplit, plot_grid_95_reactions
from load_modify_sample_utils import sample_dingo, sample_optgp, sampling_statistics, export_to_pickle
from graphs_utils import construct_graph, plot_graph
from distributions_comparison_utils import significantly_altered_reactions
from loopless_utils import get_loopless_solutions_from_samples, calculate_affected_samples, set_bounds_from_loopless_solution_samples
from loopless_utils import calculate_distances_from_samples, calculate_distances_from_reactions, violin_plot_samples_distances


/home/touliopoulos/.local/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/touliopoulos/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [3]:

#import cobra
#model = cobra.io.load_matlab_model("../private_data/Reconstructions_data/Roseburia_intestinalis_L1_82_fixed.mat")
#cobra.io.write_sbml_model(model, "MATLAB_to_SBML_Roseburia_intestinalis_L1_82_fixed.xml")

cycles_cobra_model, cycles_cobra_model_reactions, cycles_dingo_model, cycles_dingo_model_reactions = load_model("../private_data/Reconstructions_data/MATLAB_to_SBML_Roseburia_intestinalis_L1_82_fixed.xml")
#cycles_cobra_model, cycles_cobra_model_reactions, cycles_dingo_model, cycles_dingo_model_reactions = load_model("../private_data/Reconstructions_data/Roseburia_intestinalis_L1_82_AGORA2.xml")


print(len(cycles_cobra_model_reactions))

solution = cycles_cobra_model.optimize()
print(solution)


objective_functions = get_objective_functions(cycles_cobra_model)
print(objective_functions)

default_reaction_bounds = get_reaction_bounds(cycles_cobra_model)
print(default_reaction_bounds)


cycles_cobra_model_biomass_0, cycles_dingo_model_biomass_0 = modify_model(cycles_cobra_model, objective_function="biomass525", optimal_percentage=0)
cycles_cobra_model_biomass_100, cycles_dingo_model_biomass_100 = modify_model(cycles_cobra_model, objective_function="biomass525", optimal_percentage=100)



2945
<Solution 67.698 at 0x79a7d05f0250>
['biomass525']
{'1HIBUPGLU_Sabc': (0.0, 1000.0), '1HIBUP_S_GLCAASE': (0.0, 1000.0), '1HIBUP_Ste': (0.0, 1000.0), '1HMDGLUC_GLCAASE': (0.0, 1000.0), '1HMDGLUCabc': (0.0, 1000.0), '1OHMDZte': (0.0, 1000.0), '23DHMPO': (-1000.0, 1000.0), '23PDE2': (0.0, 1000.0), '23PDE4': (0.0, 1000.0), '23PDE7': (0.0, 1000.0), '23PDE9': (0.0, 1000.0), '26DAPLLAT': (-1000.0, 1000.0), '2AHBUTI': (-1000.0, 1000.0), '2HATVACIDGLUC_GLCAASE': (0.0, 1000.0), '2HATVACIDGLUCabc': (0.0, 1000.0), '2HATVACIDte': (0.0, 1000.0), '2HATVLACGLUC_GLCAASE': (0.0, 1000.0), '2HATVLACGLUCabc': (0.0, 1000.0), '2HATVLACte': (0.0, 1000.0), '2HIBUPGLU_Sabc': (0.0, 1000.0), '2HIBUP_S_GLCAASE': (0.0, 1000.0), '2HIBUP_Ste': (0.0, 1000.0), '2IMZS': (-1000.0, 1000.0), '2MBCOATA': (-1000.0, 1000.0), '2OH_CBZ_GLC_GLCAASE': (0.0, 1000.0), '2OH_CBZ_GLCabc': (0.0, 1000.0), '2OH_CBZte': (0.0, 1000.0), '2OH_MTZ_GLC_GLCAASE': (0.0, 1000.0), '2OH_MTZ_GLCabc': (0.0, 1000.0), '2OH_MTZte': (0.0, 1000.0), '

In [ ]:

samples_optgp_cycles_model = sample_optgp(cycles_cobra_model_biomass_0, n_samples = 10000, thinning = 100, reaction_in_rows = True)

samples_optgp_cycles_model_loopless_solutions = get_loopless_solutions_from_samples(samples_optgp_cycles_model, cycles_cobra_model_biomass_100)


Read LP format model from file /tmp/tmpd1d61iqg.lp
Reading time = 0.03 seconds
: 2572 rows, 6788 columns, 25528 nonzeros


/home/touliopoulos/.local/lib/python3.10/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


In [ ]:

from load_modify_sample_utils import export_to_pickle 

export_to_pickle(samples = samples_optgp_cycles_model, filename= "../private_data/Sampling_data/Virus_Samples_10K_Thinning_100_biomass525_0")
export_to_pickle(samples = samples_optgp_cycles_model_loopless_solutions, filename= "../private_data/Sampling_data/Virus_Samples_10K_Thinning_100_biomass525_0_loopless_solutions")



In [ ]:

from load_modify_sample_utils import load_from_pickle

samples_optgp_cycles_model = load_from_pickle("../private_data/Sampling_data/Virus_Samples_30K_Thinning_100_biomass525_100")
samples_optgp_cycles_model_loopless_solutions = load_from_pickle("../private_data/Sampling_data/Roseburia_Lab_Samples_30K_Thinning_100_biomass525_100_loopless_solutions")



In [6]:

distances_array = calculate_distances_from_samples(samples_optgp_cycles_model, samples_optgp_cycles_model_loopless_solutions, cycles_cobra_model_biomass_100)
violin_plot_samples_distances(distances_array)


distances_array = calculate_distances_from_reactions(samples_optgp_cycles_model, samples_optgp_cycles_model_loopless_solutions, cycles_cobra_model_biomass_100)
violin_plot_samples_distances(distances_array)
violin_plot_samples_distances(distances_array / samples_optgp_cycles_model.shape[1])


In [4]:

from optlang.symbolics import Zero, One
from optlang import Variable, Constraint, Objective

def milp_prune_model(model, reaction_ids, biomass_rxn_id, min_growth_fraction=0.1):
    """
    MILP-based approach to knock out the maximum number of reactions
    from a list while maintaining at least a minimum biomass flux.
    
    Parameters:
    - model: cobra.Model
    - reaction_ids: list of reaction IDs to consider for removal
    - biomass_rxn_id: ID of biomass reaction
    - min_growth_fraction: minimum fraction of original biomass flux to maintain
    """
    model = model.copy()
    model.solver = 'gurobi'  # or 'cplex', 'gurobi', etc.

    # Step 1: Get original growth rate
    original_solution = model.optimize()
    original_growth = original_solution.objective_value
    if original_growth is None or original_growth <= 0:
        raise ValueError("Model cannot grow initially.")

    # Step 2: Add binary variables and constraints
    binary_vars = {}
    for rxn_id in reaction_ids:
        rxn = model.reactions.get_by_id(rxn_id)
        y = Variable(f"y_{rxn_id}", type="binary")
        binary_vars[rxn_id] = y
        # Add constraint: -y*UB <= flux <= y*UB (forces flux to 0 if y=0)
        ub = rxn.upper_bound
        lb = rxn.lower_bound
        model.add_cons_vars([y])

        model.add_cons_vars([
            Constraint(rxn.flux_expression - ub * y, ub=0, name=f"ub_{rxn_id}"),
            Constraint(rxn.flux_expression - lb * y, lb=0, name=f"lb_{rxn_id}")
        ])

    # Step 3: Add minimum biomass flux constraint
    biomass_rxn = model.reactions.get_by_id(biomass_rxn_id)
    min_growth = min_growth_fraction * original_growth
    biomass_constraint = Constraint(biomass_rxn.flux_expression, lb=min_growth, name="min_biomass")
    model.add_cons_vars([biomass_constraint])

    # Step 4: Set MILP objective — maximize number of y_i that are zero (reactions removed)
    obj_expr = sum(One - binary_vars[rxn_id] for rxn_id in reaction_ids)
    model.objective = Objective(obj_expr, direction='max')

    # Step 5: Solve
    solution = model.optimize()
    if solution.status != 'optimal':
        raise RuntimeError("MILP did not converge to an optimal solution.")

    # Step 6: Extract reactions to remove
    removed_reactions = [
        rxn_id for rxn_id in reaction_ids
        if round(model.solver.variables[f"y_{rxn_id}"].primal) == 0
    ]

    return removed_reactions, solution



def remove_model_reactions(cobra_model, reactions_to_remove=[]):
                                
        cobra_model_copy = cobra_model.copy()
        
        for reaction in reactions_to_remove:
            cobra_model_copy.reactions.get_by_id(reaction).lower_bound = 0
            cobra_model_copy.reactions.get_by_id(reaction).upper_bound = 0
            
        return cobra_model_copy
    
    

In [22]:

def loopless_sampling(cobra_model, n_samples=5000, max_dist_tol=10, max_iterations=10):
    current_model = cobra_model.copy()
    current_samples = sample_optgp(cobra_model, n_samples=n_samples, thinning=100, reaction_in_rows=True)
    current_loopless = get_loopless_solutions_from_samples(current_samples, cobra_model)

    reactions_in_model = [reaction.id for reaction in current_model.reactions]
    removed_rxns = []
    
    iteration = 0
    while True:
        distances_array = calculate_distances_from_reactions(current_samples, current_loopless, current_model)
        max_dist = np.max(distances_array)
        print(f"Iteration {iteration}: max_dist = {max_dist:.4f}")

        if max_dist <= max_dist_tol:
            print("Condition met: max distance within tolerance.")
            break
        
        if iteration >= max_iterations:
            print("Max iterations reached. Stopping.")
            break

        # Identify problematic reactions
        mean_dist = np.mean(distances_array)
        std_dist = np.std(distances_array)
        mean_std = mean_dist + std_dist
        
        remove_indices = np.where(distances_array > mean_std)[0]
        reactions_in_model = [reaction.id for reaction in current_model.reactions]
        remove_reactions = [reactions_in_model[i] for i in remove_indices]

        # Prune model
        rxns, _ = milp_prune_model(current_model, remove_reactions, "biomass525", min_growth_fraction=1)
        current_model = remove_model_reactions(current_model, reactions_to_remove=rxns)

        # Re-sample and re-evaluate
        current_samples = sample_optgp(current_model, n_samples=n_samples, thinning=100, reaction_in_rows=True)
        current_loopless = get_loopless_solutions_from_samples(current_samples, current_model)

        iteration += 1
        
        removed_rxns.append(rxns)

    return current_model, removed_rxns, current_samples, current_loopless



In [23]:

(current_model, 
removed_rxns, 
current_samples, 
current_loopless) = loopless_sampling(cycles_cobra_model_biomass_100,
                                    n_samples = 5000,
                                    max_dist_tol=10,
                                    max_iterations=6)



Read LP format model from file /tmp/tmp9tq5sup5.lp
Reading time = 0.02 seconds
: 2093 rows, 5890 columns, 20352 nonzeros
Read LP format model from file /tmp/tmpai2jqg6x.lp
Reading time = 0.02 seconds
: 2093 rows, 5890 columns, 20352 nonzeros


/home/touliopoulos/.local/lib/python3.10/site-packages/cobra/util/solver.py:554: UserWarning:

Solver status is 'infeasible'.



Iteration 0: max_dist = 46376.0549
Read LP format model from file /tmp/tmp22ckb223.lp
Reading time = 0.03 seconds
: 2093 rows, 5890 columns, 20352 nonzeros
Read LP format model from file /tmp/tmp7y6hza3c.lp
Reading time = 0.02 seconds
: 2093 rows, 5890 columns, 20352 nonzeros
Read LP format model from file /tmp/tmp83po4qv6.lp
Reading time = 0.02 seconds
: 2093 rows, 5890 columns, 20352 nonzeros
Iteration 1: max_dist = 3784.0277
Read LP format model from file /tmp/tmp_eq_aya7.lp
Reading time = 0.03 seconds
: 2093 rows, 5890 columns, 20352 nonzeros
Read LP format model from file /tmp/tmp5yzu1yl4.lp
Reading time = 0.02 seconds
: 2093 rows, 5890 columns, 20352 nonzeros
Read LP format model from file /tmp/tmpo1o7iw4v.lp
Reading time = 0.02 seconds
: 2093 rows, 5890 columns, 20352 nonzeros
Iteration 2: max_dist = 884.9716
Read LP format model from file /tmp/tmpz_o6dv_n.lp
Reading time = 0.03 seconds
: 2093 rows, 5890 columns, 20352 nonzeros
Read LP format model from file /tmp/tmprt57967r.lp


In [24]:

distances_array = calculate_distances_from_samples(current_samples, current_loopless, current_model)
violin_plot_samples_distances(distances_array)

distances_array = calculate_distances_from_reactions(current_samples, current_loopless, current_model)
violin_plot_samples_distances(distances_array)


In [ ]:

removed_initial = 0
removed_final = 0

cobra_reactions_str = [str(reaction.id) for reaction in cycles_cobra_model.reactions]

for reaction in cobra_reactions_str:
    reaction_info = current_model.reactions.get_by_id(reaction)
    reaction_bounds = reaction_info.bounds
    if reaction_bounds == (0.0, 0.0):
        removed_final += 1
        
    reaction_info = cycles_cobra_model_biomass_100.reactions.get_by_id(reaction)
    reaction_bounds = reaction_info.bounds
    if reaction_bounds == (0.0, 0.0):
        removed_initial += 1
        

print(removed_final - removed_initial)


391 0
391
